# POS tagging model based on the HMM and Viterbi algorithm

## Importing libraries & the treebank corpus from nltk

In [ ]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
from copy import deepcopy
from tqdm import tqdm

#download the treebank corpus from nltk
nltk.download('treebank')

#download the universal tagset from nltk
nltk.download('universal_tagset')

# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

print(nltk_data[0])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')]


## Getting pos tags as list

In [ ]:
tags = list({tag for sent in nltk_data for word, tag in sent})
n = len(tags)
print(tags)

['DET', 'NOUN', 'ADJ', 'CONJ', 'PRT', 'ADV', 'PRON', 'X', 'ADP', 'NUM', '.', 'VERB']


## Getting rid of punctuation from pos tags (just trying new things)

In [ ]:
tags.remove('.')
n -= 1

for ind in range(len(nltk_data)):
  nltk_data[ind] = [tup for tup in nltk_data[ind] if tup[1]!='.']

## train_test_split

In [ ]:
train_set, test_set = train_test_split(nltk_data, train_size=0.8, random_state = 101)

print(train_set[2552][-3])
train_set[2552][-3] = ('I', 'NUM')
train_set[2552]

('nuclear', 'ADJ')


[('Last', 'ADJ'),
 ('year', 'NOUN'),
 ('Commonwealth', 'NOUN'),
 ('Edison', 'NOUN'),
 ('had', 'VERB'),
 ('*-1', 'X'),
 ('to', 'PRT'),
 ('refund', 'VERB'),
 ('$', '.'),
 ('72.7', 'NUM'),
 ('million', 'NUM'),
 ('*U*', 'X'),
 ('for', 'ADP'),
 ('poor', 'ADJ'),
 ('performance', 'NOUN'),
 ('of', 'ADP'),
 ('its', 'PRON'),
 ('LaSalle', 'NOUN'),
 ('I', 'NOUN'),
 ('I', 'NUM'),
 ('plant', 'NOUN'),
 ('.', '.')]

## Getting transition probabilities in **trans** DataFrame

In [ ]:
trans = pd.DataFrame(data=np.zeros((n+1, n+1)), columns=tags+['E'], index=tags+['S'])

for sent in train_set:
  prev = 'S'
  for tup in sent:
    trans.loc[prev, tup[1]] += 1
    prev = tup[1]
  trans.loc[prev, 'E'] += 1

trans = trans.divide(trans.sum(axis=1), axis=0)
trans

,DET,NOUN,ADJ,CONJ,PRT,ADV,PRON,X,ADP,NUM,.,VERB,E
DET,0.005893,0.635906,0.206411,0.000431,0.000287,0.012074,0.003306,0.045134,0.009918,0.022855,0.017393,0.040247,0.000144
NOUN,0.012976,0.262170,0.012497,0.042411,0.043935,0.016895,0.004616,0.028825,0.176740,0.009187,0.240007,0.149134,0.000610
ADJ,0.005244,0.696834,0.063313,0.016896,0.011459,0.005244,0.000194,0.020975,0.080598,0.021752,0.066032,0.011459,0.000000
CONJ,0.123491,0.349067,0.113611,0.000549,0.004391,0.057080,0.060373,0.009330,0.055982,0.040615,0.035126,0.150384,0.000000
PRT,0.101370,0.250489,0.082975,0.002348,0.001174,0.009393,0.017613,0.012133,0.019569,0.056751,0.045010,0.401174,0.000000
ADV,0.070985,0.032196,0.130721,0.006982,0.014740,0.081458,0.012025,0.022886,0.119472,0.029868,0.139255,0.339022,0.000388
PRON,0.009567,0.212756,0.070615,0.005011,0.014123,0.036902,0.006834,0.088383,0.022323,0.006834,0.041913,0.484738,0.000000
X,0.056698,0.061695,0.017682,0.010379,0.185086,0.025754,0.054200,0.075726,0.142226,0.003075,0.160869,0.206419,0.000192
ADP,0.320931,0.323462,0.107062,0.000886,0.001266,0.014553,0.069476,0.034548,0.016958,0.063275,0.038724,0.008479,0.000380
NUM,0.003569,0.351535,0.035332,0.014276,0.026053,0.003569,0.001428,0.202355,0.037473,0.184154,0.118844,0.020700,0.000714


In [ ]:
trans.sum(axis=1) # checking

DET     1.0
NOUN    1.0
ADJ     1.0
CONJ    1.0
PRT     1.0
ADV     1.0
PRON    1.0
X       1.0
ADP     1.0
NUM     1.0
.       1.0
VERB    1.0
S       1.0
dtype: float64

## Getting Emission probabilities in **emis** DataFrame

In [ ]:
vocab = list({tup[0] for sent in train_set for tup in sent})
m = len(vocab)
print(m)

emis = pd.DataFrame(data=np.zeros((m, n)), index=vocab, columns=tags)

for i, sent in enumerate(train_set):
  for tup in sent:
    emis.loc[tup] += 1

emis = emis.divide(emis.sum(axis=0),  axis=1)
emis

11052


,DET,NOUN,ADJ,CONJ,PRT,ADV,PRON,X,ADP,NUM,.,VERB
new-home,0.0,0.000000,0.000388,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000
"5,000",0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.003212,0.0,0.000000
numeral,0.0,0.000044,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000
Profit,0.0,0.000261,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000
Chiodo,0.0,0.000044,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
buy-outs,0.0,0.000131,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000
Finance,0.0,0.000174,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000
their,0.0,0.000000,0.000000,0.0,0.0,0.0,0.06697,0.0,0.0,0.000000,0.0,0.000000
total,0.0,0.000305,0.001942,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000184


In [ ]:
emis.sum(axis=0) # checking

DET     1.0
NOUN    1.0
ADJ     1.0
CONJ    1.0
PRT     1.0
ADV     1.0
PRON    1.0
X       1.0
ADP     1.0
NUM     1.0
.       1.0
VERB    1.0
dtype: float64

## Viterbi algorithm

In [ ]:
def viterbi(sent):
  n = len(tags)
  p = np.zeros(n)
  paths = [['S']]*n
  for i, word in enumerate(sent):
    paths_copy = deepcopy(paths)
    if i == 0:
      for j in range(n):
        try:
          p[j] = trans.loc['S', tags[j]] * emis.loc[word, tags[j]]
        except:
          p[j] = trans.loc['S', tags[j]]
      if i == len(sent) - 1:
        for j in range(n):
          p[j] = p[j]*trans.loc[tags[j], 'E']
          paths[j] = paths[j] + [tags[j], 'E']

    elif i == len(sent) - 1:
      for j in range(n):
        max = np.argmax(probs_till_next_word[j])
        paths[j] = paths_copy[max] + [tags[max]] + [tags[j], 'E']
        try:
          p[j] = probs_till_next_word[j][max] * emis.loc[word, tags[j]] #* trans.loc[tags[j], 'E']
        except:
          p[j] = probs_till_next_word[j][max] #* trans.loc[tags[j], 'E']
      break

    else:
      for j in range(n):
        max = np.argmax(probs_till_next_word[j])
        paths[j] = paths_copy[max] + [tags[max]]
        try:
          p[j] = probs_till_next_word[j][max] * emis.loc[word, tags[j]]
        except:
          p[j] = probs_till_next_word[j][max]
    probs_till_next_word = [[p[k] * trans.loc[tags[k], tag] for k in range(n)] for tag in tags]
    # df = pd.DataFrame(data=probs_till_next_word, index=tags, columns=tags)
    # display(df)
  return paths[np.argmax(p)], paths, p


## Checking how it works


In [ ]:
viterbi('I will eat what you eat'.split())[0][1: -1]

['PRON', 'VERB', 'VERB', 'PRON', 'PRON', 'VERB']

In [ ]:
print([trans.loc['S', 'PRON']*emis.loc['I', 'PRON']*trans.loc['PRON', 'PRON']*trans.loc['PRON', i] for i in tags])

[4.566090012868475e-07, 9.410115939563725e-06, 3.1962630090079322e-06, 3.176410443734591e-07, 6.352820887469182e-07, 1.6279103524139778e-06, 3.970513054668239e-07, 3.891102793574875e-06, 1.0918910900337656e-06, 4.1690387074016507e-07, 2.1361360234115125e-05]


In [ ]:
print([trans.loc['S', 'NUM']*emis.loc['I', 'NUM']*trans.loc['NUM', i] for i in tags])
viterbi('I love you'.split())[0][1: -1]

[4.0273174612905485e-08, 1.3598908627624419e-06, 1.409561111451692e-07, 6.443707938064878e-08, 1.0202537568602723e-07, 4.1615613766669e-08, 1.3424391537635162e-08, 7.678751959527312e-07, 1.6780489422043953e-07, 7.43711291184988e-07, 1.24846841300007e-07]


,DET,NOUN,ADJ,CONJ,PRT,ADV,PRON,X,ADP,NUM,VERB
DET,0.0,0.0,0.0,0.0,0.0,0.0,0.000050,0.0,0.0,4.027317e-08,0.0
NOUN,0.0,0.0,0.0,0.0,0.0,0.0,0.001033,0.0,0.0,1.359891e-06,0.0
ADJ,0.0,0.0,0.0,0.0,0.0,0.0,0.000351,0.0,0.0,1.409561e-07,0.0
CONJ,0.0,0.0,0.0,0.0,0.0,0.0,0.000035,0.0,0.0,6.443708e-08,0.0
PRT,0.0,0.0,0.0,0.0,0.0,0.0,0.000070,0.0,0.0,1.020254e-07,0.0
ADV,0.0,0.0,0.0,0.0,0.0,0.0,0.000179,0.0,0.0,4.161561e-08,0.0
PRON,0.0,0.0,0.0,0.0,0.0,0.0,0.000044,0.0,0.0,1.342439e-08,0.0
X,0.0,0.0,0.0,0.0,0.0,0.0,0.000427,0.0,0.0,7.678752e-07,0.0
ADP,0.0,0.0,0.0,0.0,0.0,0.0,0.000120,0.0,0.0,1.678049e-07,0.0
NUM,0.0,0.0,0.0,0.0,0.0,0.0,0.000046,0.0,0.0,7.437113e-07,0.0


,DET,NOUN,ADJ,CONJ,PRT,ADV,PRON,X,ADP,NUM,VERB
DET,3.097387e-07,0.000042,3.133269e-06,4.324155e-06,7.204173e-06,0.000015,4.566090e-07,0.000028,3.862565e-05,4.898844e-07,0.000322
NOUN,3.209757e-05,0.000309,2.469833e-04,1.222626e-05,1.771026e-05,0.000010,9.410116e-06,0.000030,3.906544e-05,1.654176e-05,0.000264
ADJ,1.045908e-05,0.000019,2.588352e-05,3.960619e-06,5.867035e-06,0.000025,3.196263e-06,0.000009,1.304203e-05,1.714595e-06,0.000160
CONJ,2.881290e-08,0.000056,7.152026e-06,1.913343e-08,1.637312e-07,0.000002,3.176410e-07,0.000010,1.061561e-07,7.838150e-07,0.000015
PRT,1.440645e-08,0.000046,4.018758e-06,1.530674e-07,8.186561e-08,0.000003,6.352821e-07,0.000079,1.516515e-07,1.241040e-06,0.000072
ADV,6.194774e-07,0.000026,2.928925e-06,2.162077e-06,7.367905e-07,0.000016,1.627910e-06,0.000013,1.834984e-06,5.062138e-07,0.000201
PRON,1.656742e-07,0.000013,9.536035e-07,2.142944e-06,1.255273e-06,0.000004,3.970513e-07,0.000025,8.462156e-06,1.632948e-07,0.000089
X,2.283422e-06,0.000035,7.696943e-06,3.252683e-07,8.459446e-07,0.000005,3.891103e-06,0.000034,4.185583e-06,9.340462e-06,0.000508
ADP,5.186322e-07,0.000195,2.908491e-05,2.123810e-06,1.419004e-06,0.000023,1.091891e-06,0.000065,2.123122e-06,2.041185e-06,0.000222
NUM,1.498271e-06,0.000016,8.786775e-06,2.142944e-06,6.003478e-06,0.000007,4.169039e-07,0.000003,1.078242e-05,9.046531e-06,0.000071


['PRON', 'VERB', 'PRON']

## Checking metrics on test_set

In [202]:
correct = 0
n_words = 0
for sent in tqdm(test_set):
  path = viterbi([tup[0] for tup in sent])[0][1:-1]
  l = len(sent)
  n_words += l
  correct +=  sum([path[i] == sent[i][1] for i in range(l)])

acc = correct/n_words
acc, correct


100%|██████████| 783/783 [00:42<00:00, 18.60it/s]


(0.9337621526072867, 19017)

In [206]:
print([tup[0] for tup in test_set[8]])
print([tup[1] for tup in test_set[8]])
print(viterbi([tup[0] for tup in test_set[8]]))

['He', 'added', ',', '``', 'This', 'has', 'nothing', '0', '*', 'to', 'do', '*T*-1', 'with', 'Marty', 'Ackerman', 'and', 'it', 'is', 'not', 'designed', ',', 'particularly', ',', '*-2', 'to', 'take', 'the', 'company', 'private', '.', "''"]
['PRON', 'VERB', '.', '.', 'DET', 'VERB', 'NOUN', 'X', 'X', 'PRT', 'VERB', 'X', 'ADP', 'NOUN', 'NOUN', 'CONJ', 'PRON', 'VERB', 'ADV', 'VERB', '.', 'ADV', '.', 'X', 'PRT', 'VERB', 'DET', 'NOUN', 'ADJ', '.', '.']
(['S', 'PRON', 'VERB', '.', '.', 'DET', 'VERB', 'NOUN', 'X', 'X', 'PRT', 'VERB', 'X', 'ADP', 'NOUN', 'NOUN', 'CONJ', 'PRON', 'VERB', 'ADV', 'VERB', '.', 'ADV', '.', 'X', 'PRT', 'VERB', 'DET', 'NOUN', 'ADJ', '.', '.', 'E'], [['S', 'PRON', 'VERB', '.', '.', 'DET', 'VERB', 'NOUN', 'X', 'X', 'PRT', 'VERB', 'X', 'ADP', 'NOUN', 'NOUN', 'CONJ', 'PRON', 'VERB', 'ADV', 'VERB', '.', 'ADV', '.', 'X', 'PRT', 'VERB', 'DET', 'NOUN', 'ADJ', '.', 'DET', 'E'], ['S', 'PRON', 'VERB', '.', '.', 'DET', 'VERB', 'NOUN', 'X', 'X', 'PRT', 'VERB', 'X', 'ADP', 'NOUN', 'NO